In [1]:
import numpy as np
import pandas as pd
from scipy import fftpack
from scipy import signal
import time
import os
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import pylab
import pickle
import copy
import processing
import matplotlib.animation as animation
import datetime
import multiprocessing as mp
import processing
from scipy import stats

%matplotlib inline

共通変数をここに定義

In [2]:
DataName='sensor3_AccX'
SensorName='sensor3'
Axis='AccX'
SampleNum=16
width=1000

In [3]:
WORKSPACE_PATH = '/home/takeyama/pywork/ipython/2016-07-11'

StudyDataPath=WORKSPACE_PATH+'/Study/'+DataName+'/'
if not os.path.exists(StudyDataPath): os.makedirs(StudyDataPath)

GlaphDataPath=WORKSPACE_PATH+'/Graph/'+DataName+'/kullback/'
if not os.path.exists(GlaphDataPath): os.makedirs(GlaphDataPath)

StudyDataModelPicPath=WORKSPACE_PATH+'/Study/'+DataName+'/modelPic/'
if not os.path.exists(StudyDataModelPicPath): os.makedirs(StudyDataModelPicPath)

WindowDataPath=WORKSPACE_PATH+'/window/'
RawDataPath=WORKSPACE_PATH+'/raw/'

In [4]:
"""
    encordData,decordData
    firstNum:
    goalNum:
    commonLabel:共通名前
"""
def MakeGlaph(encordData,decordData,firstNum,goalNum,commonLabel):
    eRow, eCol=encordData.shape
    dRow, dCol=decordData.shape
    
    if eRow!=dRow | eCol!=dCol:
        return -1
    start = firstNum
    goal=goalNum
    print str(start)+' is start '+str(goal)+' is goal'
    dataE = encordData.reshape(eRow*eCol,1)[start:goal]
    dataD = decordData.reshape(dRow*dCol,1)[start:goal]
    plt.clf() 
    plt.figure(figsize=(100,50))
    plt.plot(range(0,len(dataE)),dataE,'--r',label="raw data")
    plt.plot(range(0,len(dataD)),dataD,'b',label="reconstruct")
    plt.legend(loc='best')
    plt.title(str(start)+"-"+str(goal))
    plt.savefig(commonLabel+'-'+str(start)+"-"+str(goal)+'-'+'glaph')
    plt.clf()
    plt.cla()
    plt.close('all')
    return 1

すべてのデータを読みdし、グラフ化する

並列処理でグラフを作成する

In [5]:
encoding_dim = 4
PowerDataE=processing.LoadDicDataFromFileNPZ(StudyDataPath+DataName+'_edim='+str(encoding_dim)+'-power-raw.npz')
PowerDataD=processing.LoadDicDataFromFileNPZ(StudyDataPath+DataName+'_edim='+str(encoding_dim)+'-power-restruct.npz')

# to distribution
DistributePowerDataE = np.array([])
DistributePowerDataD = np.array([])

for i in range(PowerDataE.shape[0]):
    DistributePowerDataE = np.append(DistributePowerDataE,np.sum(PowerDataE[i,:]))


In [6]:
PowerDataE

array([[  5.36123771e-04,   8.13515949e-04,   2.77084325e-04, ...,
          1.84304051e-04,   6.02657985e-06,   2.02500000e-05],
       [  1.63592263e-03,   4.06826047e-04,   1.76972059e-04, ...,
          3.90739526e-05,   4.10430352e-05,   2.60100000e-05],
       [  3.27490404e-03,   5.96397092e-05,   1.66218818e-04, ...,
          3.14402908e-05,   6.77613764e-05,   4.62400000e-05],
       ..., 
       [  1.81787426e-01,   4.54255692e-02,   2.19838776e-02, ...,
          7.79657076e-03,   7.84752517e-03,   5.91361000e-03],
       [  2.52707265e-01,   4.95676245e-02,   2.01651677e-02, ...,
          8.93633549e-03,   8.20966959e-03,   7.46496000e-03],
       [  1.02175587e-01,   3.60586535e-02,   2.53537079e-02, ...,
          6.07896648e-03,   5.91213631e-03,   6.06841000e-03]])

In [7]:
PowerDataE.shape

(15921, 8)

In [8]:
PowerDataE[0,:]

array([  5.36123771e-04,   8.13515949e-04,   2.77084325e-04,
         1.20530000e-04,   1.44405325e-04,   1.84304051e-04,
         6.02657985e-06,   2.02500000e-05])

In [9]:
np.sum(PowerDataE[0,:])

0.0021022400000000195

In [10]:
PowerDataE[0]/( np.sum(PowerDataE[0]) )

array([ 0.25502501,  0.38697577,  0.13180433,  0.05733408,  0.06869117,
        0.08767032,  0.00286674,  0.00963258])

In [11]:
DistributePowerDataE = np.array([])
for i in range( PowerDataE.shape[0] ):
    DistributePowerDataE = np.append(DistributePowerDataE,PowerDataE[0]/( np.sum(PowerDataE[0]) ))

In [12]:
DistributePowerDataE = DistributePowerDataE.reshape(PowerDataE.shape)

In [13]:
DistributePowerDataE.shape

(15921, 8)

In [14]:
# define SaveFileName
# 8, 4, 12, 16, 32, 64
# 32 floats -> compression of factor 24.5, assuming the input is 784 floats
def kld(p, q):
    """Calculates Kullback–Leibler divergence"""
    p = np.array(p)
    q = np.array(q)
    return np.sum(p * np.log(p / q), axis=(p.ndim - 1))
 
def jsd(p, q):
    """Calculates Jensen-Shannon Divergence"""
    p = np.array(p)
    q = np.array(q)
    m = 0.5 * (p + q)
    return 0.5 * kld(p, m) + 0.5 * kld(q, m)

AllEncoding_dim = [4, 8, 12, 16, 32, 64]  
window = processing.LoadDicDataFromFileNPZ(WindowDataPath+SensorName+'_'+Axis+'_train_edim=8.npz')
fig = plt.figure(figsize=(30,15))
ax = fig.add_subplot(111)

for encoding_dim in AllEncoding_dim:
    SaveFileNameDecord=DataName+'_decoded'+'_edim='+str(encoding_dim)
    SaveFileNameGraph=GlaphDataPath+ DataName+'_edim='+str(encoding_dim)

    reconstructData = processing.LoadDicDataFromFileNPZ(StudyDataPath+SaveFileNameDecord)
    
    PowerDataE=processing.LoadDicDataFromFileNPZ(StudyDataPath+DataName+'_edim='+str(encoding_dim)+'-power-raw.npz')
    PowerDataD=processing.LoadDicDataFromFileNPZ(StudyDataPath+DataName+'_edim='+str(encoding_dim)+'-power-restruct.npz')

    # to distribution
    DistributePowerDataE = np.array([])
    DistributePowerDataD = np.array([])

    for i in range( PowerDataE.shape[0] ):
        DistributePowerDataE = np.append(DistributePowerDataE,PowerDataE[i]/( np.sum(PowerDataE[i]) ))
    DistributePowerDataE = DistributePowerDataE.reshape(PowerDataE.shape)
    
    for i in range( PowerDataD.shape[0] ):
        DistributePowerDataD = np.append(DistributePowerDataD,PowerDataD[i]/( np.sum(PowerDataD[i]) ))
    DistributePowerDataD = DistributePowerDataD.reshape(PowerDataD.shape)
    
    processing.SaveDicDataFromFileNPZ(StudyDataPath, DataName+'_edim='+str(encoding_dim)+'-power-distribution-raw',DistributePowerDataE)
    processing.SaveDicDataFromFileNPZ(StudyDataPath, DataName+'_edim='+str(encoding_dim)+'-power-distribution-reconst',DistributePowerDataD)
    
    ArrayKLD = np.array([])
    
    # Kullback-leibler-divergence plot
    for i in range(DistributePowerDataE.shape[0]):
        ArrayKLD = np.append(ArrayKLD, kld(DistributePowerDataE[i],DistributePowerDataD[i]))

    for i in range(width,DistributePowerDataE.shape[0],width):
        start =i-width
        goal=i

        data = ArrayKLD[start:goal]

        #dataE,dataDをPower
        ax.plot(range(0,len(data)),data,'g',label="kld data")
        ax.legend(loc='best')
        ax.set_title(str(start)+"-"+str(goal))
        ax.figure.savefig(SaveFileNameGraph+'-'+str(start)+"-"+str(goal)+'-'+'-glaph')
        ax.clear()
        print 'Graph range='+str(start)+' : '+str(goal)
        time.sleep(3)

IOError: [Errno 2] No such file or directory: '/home/takeyama/pywork/ipython/2016-07-11/window/sensor3_AccX_train_edim=8.npz'

In [ ]:
window.size